In [1]:
# not needed - package were installed using pip in terminal with python3.11.5 (kernel used here)
#! pip install matplotlib
#! pip install xarray
#! pip install xhistogram
#! pip install h5netcdf
#! pip install scipy
#! pip install netCDF4-python

In [1]:

import importlib
import matplotlib.pyplot as plt
import numpy as np
import xrUWLCM as xrU


#from xhistogram.xarray import histogram
import xarray as xr

In [2]:
pgsdir = '/net/pr2/projects/plgrid/plgguwicmw/wyniki/icmw24_cc/'
figoutdir = '/net/people/plgrid/plgpdziekan/wyniki/icmw24_cc/figs/'
ncoutdir = '/net/people/plgrid/plgpdziekan/wyniki/icmw24_cc/netcdf/'

datadir = {}

datadir['regular'] = {}
datadir['regular']['50m'] = {}
datadir['regular']['50m']['SD128'] = pgsdir + "14_08_2024_out_UWLCM_icmw24_cc_regular_SMG_res50m_sd128_mpi1x8_out_lgrngn/" 

datadir['clean'] = {}

datadir['polluted'] = {}

datadir['regular_noGCCN'] = {}
datadir['regular_noGCCN']['50m'] = {}
#datadir['regular_noGCCN']['50m_SD16'] = pgsdir + "12_08_2024_out_UWLCM_icmw24_cc_SurfFix_SMG_res50m_sd16_mpi1x8_out_lgrngn/" 
datadir['regular_noGCCN']['50m']['SD128'] = pgsdir + "12_08_2024_out_UWLCM_icmw24_cc_regular_noGCCN_SMG_res50m_sd128_mpi1x8_out_lgrngn/" 


datadir['regular_ILES'] = {}
datadir['regular_ILES']['50m'] = {}
datadir['regular_ILES']['50m']['SD128'] = pgsdir + "20_08_2024_out_UWLCM_icmw24_cc_regular_ILES_res50m_sd128_mpi1x8_out_lgrngn/" 


In [3]:
%%time

#aerosol_to_plot='clean'
#aerosol_to_plot='regular'
#aerosol_to_plot='polluted'
#aerosol_to_plot='regular_noGCCN'
aerosol_to_plot='regular_ILES'
resolution_to_plot='50m'

#aerosol_to_plot='regular_thprtrb'

#datadir['test'] = '/home/piotr/praca/code/UWLCM/build/outdir/'
data = {}
data_DSD = {}
for sdname,dir in datadir[aerosol_to_plot][resolution_to_plot].items():
    data[sdname], data_DSD[sdname] = xrU.load_outdir(dir)
    data[sdname] = xrU.calc_all(data[sdname]).pipe(xrU.convert_units)
    data[sdname] = xrU.calc_cloud_base(data[sdname], data[sdname].rc>0.1)
    data[sdname] = xrU.calc_cloud_top(data[sdname], data[sdname].rc>0.1)
    #data_DSD[sdname] = xrU.calc_all(data_DSD[sdname]).pipe(xrU.convert_units)

Group sgs not found in /net/pr2/projects/plgrid/plgguwicmw/wyniki/icmw24_cc/20_08_2024_out_UWLCM_icmw24_cc_regular_ILES_res50m_sd128_mpi1x8_out_lgrngn/const.h5
Group user_params not found in /net/pr2/projects/plgrid/plgguwicmw/wyniki/icmw24_cc/20_08_2024_out_UWLCM_icmw24_cc_regular_ILES_res50m_sd128_mpi1x8_out_lgrngn/const.h5
CPU times: user 7.28 s, sys: 854 ms, total: 8.13 s
Wall time: 19.5 s


/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in sqrt
  return func(*(_execute_task(a, cache) for a in args))
/net/people/plgrid/plgpdziekan/.local/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: invalid value encoun

In [5]:
data['100m_SD32'].to_netcdf("/home/piotr/praca/ICMW2024_CC/test")

KeyError: '100m_SD32'

In [ ]:
#data['100m_blk1m']
data['100m_SD32']

In [ ]:
data['100m_blk1m'].w.mean(["x","y","z"]).plot(label='thprtrb', xlim=(60,None))
dataregular, dataregular_DSD = xrU.load_outdir(datadir['regular']['100m_SD32'])
dataregular = xrU.calc_all(dataregular).pipe(xrU.convert_units)
dataregular = xrU.calc_cloud_base(dataregular, dataregular.qc>0.1)
dataregular = xrU.calc_cloud_top(dataregular, dataregular.qc>0.1)
dataregular.w.mean(["x","y","z"]).plot(label='normal', xlim=(60,None))
plt.legend()

## CFADs

In [6]:
%%time
#CFADs

plot_parameters = {
    'rc': dict(xlim=(0,15), ylim=(0,7000), nicename='$q_c$', units='[g/kg]', xscale='linear', kwargs=dict( vmin=0, vmax=1.0)),
    'rr': dict(xlim=(1e-10,1.2e1), ylim=(0,7000), nicename='$q_r$', units='[g/kg]', xscale='log', kwargs=dict( vmin=0, vmax=0.2)),
    'r_mean': dict(xlim=(0,18), ylim=(0,7000), nicename='$<r>$', units='[$\mu m$]', xscale='linear', kwargs=dict( vmin=0, vmax=0.3)),
    'r_sigma': dict(xlim=(0,9), ylim=(0,7000), nicename='$\sigma(r)$', units='[$\mu m$]', xscale='linear', kwargs=dict( vmin=0, vmax=0.2)),
    'w': dict(xlim=(-12,20), ylim=(0,7000), nicename='$w$', units='[m/s]', xscale='linear', kwargs=dict( vmin=0, vmax=0.25)),
    'r_m6': dict(xlim=(1e-6,1e4), ylim=(0,7000), nicename='$r^6$', units='[$mm^6/m^3$]', xscale='log', kwargs=dict( vmin=0, vmax=0.5)),
    'na': dict(xlim=(0,1e3), ylim=(0,7000), nicename='$N_a$', units='[$mg^{-1}$]', xscale='linear', kwargs=dict( vmin=0, vmax=1)),
    'nc': dict(xlim=(0,1.5e3), ylim=(0,7000), nicename='$N_c$', units='[$mg^{-1}$]', xscale='linear', kwargs=dict( vmin=0, vmax=0.2)),
    'nr': dict(xlim=(1e-10,1e3), ylim=(0,7000), nicename='$N_r$', units='[$mg^{-1}$]', xscale='log', kwargs=dict( vmin=0, vmax=0.25)),
}


for sdname in data:
    _data = data[sdname]
    
    for varname, params in plot_parameters.items():
        #xrU.plot_CFAD(_data, _data[varname].where(_data.qc>0.1), xbins=np.linspace(params['xmin'], params['xmax'], 100), ylim=(0,7000), **params)=np.linspace(params['xmin'], params['xmax'], 100), ylim=(0,7000), **params)
        if params['xscale']=='linear':
            xbins=np.linspace(params['xlim'][0], params['xlim'][1], 100)
        else:
            xbins=np.logspace(np.log10(params['xlim'][0]), np.log10(params['xlim'][1]), 100)
        
        xrU.plot_CFAD(_data, varname, _data.rc>0.1, xbins=xbins, **params['kwargs'])
        
        plt.xlim(params['xlim'])
        plt.ylim(params['ylim'])
        plt.xlabel(params['nicename'] + ' ' + params['units'])
        plt.legend('',frameon=False)
        plt.title('CFAD of ' + params['nicename'] + ' in cloudy cells')
        plt.xscale(params['xscale'])
        #xbins=np.linspace(params['extent'][0], params['extent'][1], 100),
        #xrU.plot_DSD(data, data.qc>0.1, label=str(sdname))
        #xrU.plot_DSD(data, label=str(sdname))
        
        #plt.show()
        plt.savefig(figoutdir + "/CFAD/"+str(aerosol_to_plot)+"/res"+str(resolution_to_plot)+"_"+varname+"_"+sdname+"_"+str(aerosol_to_plot)+"_CFAD.png", dpi=300)
        plt.clf()
        
    
    _data.close()
        

CPU times: user 2min 52s, sys: 44.6 s, total: 3min 36s
Wall time: 1min 28s


<Figure size 640x480 with 0 Axes>

## DSDs

In [7]:
%%time
#DSD

for z in [1000,2000,3000]:
    for sdname in data:
        data_DSD_z = data_DSD[sdname].sel(z=z, method='nearest')
        data_z = data[sdname].sel(z=z, method='nearest')
        xrU.plot_DSD(data_DSD_z, data_z.rc>0.1, label=str(sdname))
        #xrU.plot_DSD(data_DSD_z, label=str(sdname))
        
    plt.xscale("log")
    plt.yscale("log")
    plt.xlim(left=0.75)
    plt.title("DSD in cloudy cells at " + str(z) + " m")
    plt.xlabel("radius [um]")
    plt.ylabel('concentration density [ 1 / cc / log(um)]')
    #plt.yscale("log")
    plt.legend()
    #plt.show()
    #plt.savefig("./ICMW24_CC_UWLCM_results/DSD/res100m_z"+str(z)+"m_DSD.png")
    plt.savefig(figoutdir + "/DSD/res"+str(resolution_to_plot)+"_z"+str(z)+"m_"+str(aerosol_to_plot)+"_DSD.png", dpi=300)
    plt.clf()
    

CPU times: user 1min 13s, sys: 52.9 s, total: 2min 5s
Wall time: 2min 13s


<Figure size 640x480 with 0 Axes>

## TIME SERIES

In [4]:
%%time

plot_parameters = {
    'rl_i': dict(nicename='total liquid water', units='[kg]', reduce='3D_sum'),
    'rc_i': dict(nicename='total cloud water', units='[kg]', reduce='3D_sum'),
    'rr_i': dict(nicename='total rain water', units='[kg]', reduce='3D_sum'),
    'rt_i': dict(nicename='total water', units='[kg]', reduce='3D_sum'),
    'cb_z': dict(nicename='cloud base', units='[m]', reduce='2D_min'),
    'ct_z': dict(nicename='cloud top', units='[m]', reduce='2D_max'),
    'prflux_s': dict(nicename='surface precipitation flux', units='[W/m$^2$]', reduce='2D_mean'),
    'prflux_cb': dict(nicename='cloud base (700m) precipitation flux', units='[W/m$^2$]', reduce='2D_mean'),
    'acc_precip_s': dict(nicename='accumulated surface precipitation', units='[mm]', reduce='none'),
    'acc_precip_cb': dict(nicename='accumulated cloud base (700m) precipitation', units='[mm]', reduce='2D_sum'),
}

for varname, params in plot_parameters.items():
    for sdname in data:
        # some vars need to be calculated (TODO: move it to calculate.py?)
        if varname == 'prflux_s':
            _data=data[sdname]['prflux'].sel(z=0, method='nearest')
        elif varname == 'prflux_cb':
            #_data=data[sdname]['prflux'].sel(z=data[sdname]['cb_z'].min(["x","y"]), method='nearest')
            _data=data[sdname]['prflux'].sel(z=700, method='nearest')
        elif varname == 'acc_precip_s':
            _data=data[sdname]['puddle_liquid_volume']/data[sdname]['surf_area']*1e3
        elif varname == 'acc_precip_cb':
            #_data=data[sdname]['prflux'].sel(z=data[sdname]['cb_z'].min(["x","y"]), method='nearest') \
            #    / xrU.L_evap * data[sdname].outfreq * data[sdname].dt
            _data=data[sdname]['prflux'].sel(z=700, method='nearest') \
                / xrU.L_evap * data[sdname].outfreq * data[sdname].dt
            _data=_data.cumsum()
        elif varname == 'rt_i':
            _data=data[sdname]['rt'] * data[sdname]['rhod'] * data[sdname]['dv'] / 1e3
        elif varname == 'rl_i':
            _data=data[sdname]['rl'] * data[sdname]['rhod'] * data[sdname]['dv'] / 1e3
        elif varname == 'rc_i':
            _data=data[sdname]['rc'] * data[sdname]['rhod'] * data[sdname]['dv'] / 1e3
        elif varname == 'rr_i':
            _data=data[sdname]['rr'] * data[sdname]['rhod'] * data[sdname]['dv'] / 1e3
        else:
            _data=data[sdname][varname]
            
        if params['reduce']=='3D_mean':
            res=_data.mean(["x","y","z"])
        elif params['reduce']=='3D_sum':
            res=_data.sum(["x","y","z"])
        elif params['reduce']=='2D_mean':
            res=_data.mean(["x","y"])
        elif params['reduce']=='2D_sum':
            res=_data.sum(["x","y"])
        elif params['reduce']=='2D_min':
            res=_data.min(["x","y"])
        elif params['reduce']=='2D_max':
            res=_data.max(["x","y"])
        elif params['reduce']=='none':
            res=_data
        res.to_dataset(name=varname).to_netcdf(ncoutdir+"/series/res"+str(resolution_to_plot)+"_"+str(aerosol_to_plot)+"_series.nc", group=sdname, mode='a', engine='h5netcdf')
        res.plot(xlim=(6000,None), label=sdname)
    

    
    plt.legend()
    plt.xlabel('t [s]')
    plt.ylabel(params['nicename']+' '+params['units'])
    plt.title('')
    #plt.show()
    plt.savefig(figoutdir+"/series/res"+str(resolution_to_plot)+"_"+str(varname)+"_"+str(aerosol_to_plot)+"_series.png", dpi=300)
    plt.clf()

CPU times: user 25.6 s, sys: 5.94 s, total: 31.6 s
Wall time: 23.7 s


<Figure size 640x480 with 0 Axes>

In [ ]:
_data.qc.sel(t=7300, method='nearest').mean(["y"]).plot(x="x")

In [ ]:
_data.sd_conc.mean(["x","y","z"]).plot()

In [ ]:
_data.puddle_water_number.plot()

In [ ]:
#_data.sd_conc.sel(t=1, method='nearest').values
_data

In [ ]:
_data.nr.where(_data.t<8000).where(_data.qc>0.1).mean(["x","y","t"]).plot(y="z",ylim=(0,6000))

In [ ]:
data.w.where(data.t<8000).where(data.qc>0.1).mean(["x","y","t"]).plot(y="z",ylim=(0,6000))

In [ ]:
data.na.where(data.t<8000).where(data.qc>0.1).mean(["x","y","t"]).plot(y="z",ylim=(0,6000))

In [ ]:
data.nr.where(data.t<8000).where(data.qc>0.1).mean(["x","y","t"]).plot(y="z",ylim=(0,6000))

In [ ]:
data.cloud_rw_mom0.where(data.t<8000).where(data.cloud_rw_mom3>1e-7).mean(["x","y","t"]).plot(y="z",ylim=(0,6000))

In [ ]:
data.puddle_liquid_volume.plot()

In [ ]:
data